In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [2]:
def plot_prediction(train_main_series, forecast_series, show_last_n =60 ):
    
    fig = go.Figure()


    fig.add_trace(go.Scatter(x=train_main_series.index[-show_last_n:], y=train_main_series.iloc[-show_last_n:],
                        mode='lines',
                        name='History'))
        
    fig.add_trace(go.Scatter(x = forecast_series.index, y = forecast_series.values,
                        mode='lines',
                        name='Predict'))
    
    fig.update_layout(
        autosize=False,
        width=500,
        height=500,
    )
    return fig


In [3]:
def get_yfin_data(ticker, start_date = None, end_date = None):
    resultData = yf.download(ticker, start_date, end_date)

    resultData = resultData[["Open", "High","Low", "Close", "Adj Close", "Volume"]]
    return resultData

In [4]:
startDate = '2015-03-01'


ticker = 'AAPL'



train_main_series = get_yfin_data(ticker,startDate)

[*********************100%%**********************]  1 of 1 completed


Sarima

In [6]:
from sklearn.metrics import mean_absolute_percentage_error  as mape
from sklearn.metrics import mean_absolute_error  as mae
from sklearn.metrics import mean_squared_error  as mse

def get_metrics(y_true, y_pred):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    res = {}
    res['MAE'] = mae(y_true, y_pred)
    res['MAPE'] = mape(y_true, y_pred)
    res['RMSE'] = mse(y_true, y_pred)**(1/2)
    return res

In [7]:
#def Arima_model(train_main_series, horizon, p = 1,d = 1,q = 1):
from statsmodels.tsa.statespace.sarimax import SARIMAX
p,d,q = 3,2,2
horizon = 20

model = SARIMAX(train_main_series['Close'], order =(p,d,q),enforce_invertibility=True,enforce_stationarity=True).fit()
predict_numpy = model.forecast(steps = horizon).values


def evaluate_sarima(data, steps = [60, 30, 14, 7],p = 2,d = 2,q = 2):
    metrics_df = pd.DataFrame(index = ['MAE', 'MAPE', 'RMSE'])
    for step in steps:
        fitted_model = SARIMAX(data[:-step]['Close'], order = (p,d,q)).fit()
        forecast = fitted_model.forecast(step)
        metrics_df[step] = get_metrics(data[-step:]['Close'], forecast).values()
    return metrics_df

metrics_df = evaluate_sarima(data = train_main_series,p = p, d = d, q = q)
forecast_series = pd.Series(data = predict_numpy, index=pd.date_range(train_main_series.index[-1], periods=horizon))
fig = plot_prediction(train_main_series['Close'],forecast_series)


c:\Users\Dron\anaconda3\envs\doma\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Dron\anaconda3\envs\doma\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Dron\anaconda3\envs\doma\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Dron\anaconda3\envs\doma\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.


In [8]:
plot_prediction(train_main_series['Close'],forecast_series)